# Audio Cleaning
With my handy new legend, I can easily go and select all the audio clips I want. I’ll be applying a voice activity detection filter (VAD) on each clip to help remove some of the background sound.

In [1]:
# Audio Modules
import collections
import contextlib
import sys
import wave
import webrtcvad
import sounddevice as sd
# General Modules
import numpy as np
import os
import pandas as pd
import zipfile

# Functions
Audio processing in python is a relativley new thing for me so I "found inspiration" (aka tweeked snippets of code) from the projects of others.
- VAD by [John Wiseman](https://github.com/wiseman)
- Byte to numpy array conversion by [HudsonHuang](https://gist.github.com/HudsonHuang)

In [2]:
def read_wave(path):
    """Reads a .wav file.
    Takes the path, and returns (PCM audio data, sample rate).
    """
    with contextlib.closing(wave.open(path, 'rb')) as wf:
        num_channels = wf.getnchannels()
        assert num_channels == 1
        sample_width = wf.getsampwidth()
        assert sample_width == 2
        sample_rate = wf.getframerate()
        assert sample_rate in (8000, 16000, 32000, 48000)
        pcm_data = wf.readframes(wf.getnframes())
        return pcm_data, sample_rate

def write_wave(path, audio, sample_rate):
    """Writes a .wav file.
    Takes path, PCM audio data, and sample rate.
    """
    with contextlib.closing(wave.open(path, 'wb')) as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(sample_rate)
        wf.writeframes(audio)

class Frame(object):
    """Represents a "frame" of audio data."""
    def __init__(self, bytes, timestamp, duration):
        self.bytes = bytes
        self.timestamp = timestamp
        self.duration = duration

def frame_generator(frame_duration_ms, audio, sample_rate):
    """Generates audio frames from PCM audio data.
    Takes the desired frame duration in milliseconds, the PCM data, and
    the sample rate.
    Yields Frames of the requested duration.
    """
    n = int(sample_rate * (frame_duration_ms / 1000.0) * 2)
    offset = 0
    timestamp = 0.0
    duration = (float(n) / sample_rate) / 2.0
    while offset + n < len(audio):
        yield Frame(audio[offset:offset + n], timestamp, duration)
        timestamp += duration
        offset += n

def vad_collector(sample_rate, frame_duration_ms,
                  padding_duration_ms, vad, frames):
    """Filters out non-voiced audio frames.
    Given a webrtcvad.Vad and a source of audio frames, yields only
    the voiced audio.
    Uses a padded, sliding window algorithm over the audio frames.
    When more than 90% of the frames in the window are voiced (as
    reported by the VAD), the collector triggers and begins yielding
    audio frames. Then the collector waits until 90% of the frames in
    the window are unvoiced to detrigger.
    The window is padded at the front and back to provide a small
    amount of silence or the beginnings/endings of speech around the
    voiced frames.
    Arguments:
    sample_rate - The audio sample rate, in Hz.
    frame_duration_ms - The frame duration in milliseconds.
    padding_duration_ms - The amount to pad the window, in milliseconds.
    vad - An instance of webrtcvad.Vad.
    frames - a source of audio frames (sequence or generator).
    Returns: A generator that yields PCM audio data.
    """
    num_padding_frames = int(padding_duration_ms / frame_duration_ms)
    # We use a deque for our sliding window/ring buffer.
    ring_buffer = collections.deque(maxlen=num_padding_frames)
    # We have two states: TRIGGERED and NOTTRIGGERED. We start in the
    # NOTTRIGGERED state.
    triggered = False

    voiced_frames = []
    for frame in frames:
        is_speech = vad.is_speech(frame.bytes, sample_rate)

        #sys.stdout.write('1' if is_speech else '0')
        if not triggered:
            ring_buffer.append((frame, is_speech))
            num_voiced = len([f for f, speech in ring_buffer if speech])
            # If we're NOTTRIGGERED and more than 90% of the frames in
            # the ring buffer are voiced frames, then enter the
            # TRIGGERED state.
            if num_voiced > 0.9 * ring_buffer.maxlen:
                triggered = True
                # sys.stdout.write('+(%s)' % (ring_buffer[0][0].timestamp,))
                # We want to yield all the audio we see from now until
                # we are NOTTRIGGERED, but we have to start with the
                # audio that's already in the ring buffer.
                for f, s in ring_buffer:
                    voiced_frames.append(f)
                ring_buffer.clear()
        else:
            # We're in the TRIGGERED state, so collect the audio data
            # and add it to the ring buffer.
            voiced_frames.append(frame)
            ring_buffer.append((frame, is_speech))
            num_unvoiced = len([f for f, speech in ring_buffer if not speech])
            # If more than 90% of the frames in the ring buffer are
            # unvoiced, then enter NOTTRIGGERED and yield whatever
            # audio we've collected.
            if num_unvoiced > 0.9 * ring_buffer.maxlen:
                #sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
                triggered = False
                yield b''.join([f.bytes for f in voiced_frames])
                ring_buffer.clear()
                voiced_frames = []
    #if triggered:
    #    sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
    #sys.stdout.write('\n')
    # If we have any leftover voiced audio when we run out of input,
    # yield it.
    if voiced_frames:
        yield b''.join([f.bytes for f in voiced_frames])

def VAD(path,ag=1):
    audio, sample_rate = read_wave(path)
    vad = webrtcvad.Vad(int(ag))
    frames = frame_generator(30, audio, sample_rate)
    frames = list(frames)
    segments = vad_collector(sample_rate, 30, 300, vad, frames)
    
    audio_byte = b"".join(segments)
    return (audio_byte,sample_rate)

def byte_to_float(byte):
    # byte -> int16(PCM_16) -> float32
    return pcm2float(np.frombuffer(byte,dtype=np.int16), dtype='float32')

def pcm2float(sig, dtype='float32'):
    """Convert PCM signal to floating point with a range from -1 to 1.
    Use dtype='float32' for single precision.
    Parameters
    ----------
    sig : array_like
        Input array, must have integral type.
    dtype : data type, optional
        Desired (floating point) data type.
    Returns
    -------
    numpy.ndarray
        Normalized floating point data.
    See Also
    --------
    float2pcm, dtype
    """
    sig = np.asarray(sig)
    if sig.dtype.kind not in 'iu':
        raise TypeError("'sig' must be an array of integers")
    dtype = np.dtype(dtype)
    if dtype.kind != 'f':
        raise TypeError("'dtype' must be a floating point type")

    i = np.iinfo(sig.dtype)
    abs_max = 2 ** (i.bits - 1)
    offset = i.min + abs_max
    return (sig.astype(dtype) - offset) / abs_max

# Collection and VAD
Grab the audio clips and apply VAD.

In [3]:
legend = pd.read_csv('Data/CSVs/Audio Legend Raw.csv',index_col='Unnamed: 0')
legend.head()

,path,original,statment,emotion,sex
0,Data/Raw Audio/CREMA_D/,1001_DFA_ANG_XX.wav,3,anger,Male
1,Data/Raw Audio/CREMA_D/,1001_DFA_DIS_XX.wav,3,disgust,Male
2,Data/Raw Audio/CREMA_D/,1001_DFA_FEA_XX.wav,3,fear,Male
3,Data/Raw Audio/CREMA_D/,1001_DFA_HAP_XX.wav,3,happy,Male
4,Data/Raw Audio/CREMA_D/,1001_DFA_NEU_XX.wav,3,neutral,Male


In [4]:
# Dataframe to save everything in
audio = pd.DataFrame()
# A list to put any filenames that run into errors
error = []

count=1
for index in legend.index:
    try:
        entry = legend.loc[index]
        track = entry['original']
        path = entry['path']
        audio_byte,rate = VAD(path+track)
        audio = audio.append([[track,audio_byte,rate]])
    except:
        error.append(track)
    # progress bar
    print('%4d of %d complete (%d%%)' % (count,len(legend),(count/len(legend))*100),end='\r')
    count+=1
    
audio.columns = ['original','audio','sample rate']
audio.head()

,original,audio,sample rate
0,1001_DFA_ANG_XX.wav,b'\xae\x00\xa9\x00\xb1\x00\xac\x00\xa8\x00\xad...,16000
0,1001_DFA_DIS_XX.wav,b'C\x00\xf3\xff\xdb\xff]\x00\xa8\x00n\x00\x1b\...,16000
0,1001_DFA_FEA_XX.wav,b'\xd2\x00\xa9\x00\xf1\x00\x8d\x00\xe5\x00\xf6...,16000
0,1001_DFA_HAP_XX.wav,b'4\xffP\xff\x0e\xff\xb8\xfe\xb3\xfe\'\xff1\xf...,16000
0,1001_DFA_NEU_XX.wav,b'\xe5\xfe\xf4\xfe\xef\xfe\x06\xff\x19\xff\x14...,16000


# Cleaning
Merge audio with the legend and drop any clips that seem to be empty

In [5]:
data = legend.merge(audio,on='original')
data.head()

,path,original,statment,emotion,sex,audio,sample rate
0,Data/Raw Audio/CREMA_D/,1001_DFA_ANG_XX.wav,3,anger,Male,b'\xae\x00\xa9\x00\xb1\x00\xac\x00\xa8\x00\xad...,16000
1,Data/Raw Audio/CREMA_D/,1001_DFA_DIS_XX.wav,3,disgust,Male,b'C\x00\xf3\xff\xdb\xff]\x00\xa8\x00n\x00\x1b\...,16000
2,Data/Raw Audio/CREMA_D/,1001_DFA_FEA_XX.wav,3,fear,Male,b'\xd2\x00\xa9\x00\xf1\x00\x8d\x00\xe5\x00\xf6...,16000
3,Data/Raw Audio/CREMA_D/,1001_DFA_HAP_XX.wav,3,happy,Male,b'4\xffP\xff\x0e\xff\xb8\xfe\xb3\xfe\'\xff1\xf...,16000
4,Data/Raw Audio/CREMA_D/,1001_DFA_NEU_XX.wav,3,neutral,Male,b'\xe5\xfe\xf4\xfe\xef\xfe\x06\xff\x19\xff\x14...,16000


In [6]:
# check for any empty tracks
empty_tracks = data[data['audio'] == b""]
print('There are currently',len(empty_tracks),'empty tracks')
# and drop them
data = data.drop(empty_tracks.index)
print('There are now',len(data[data['audio'] == b""]),'empty tracks')

There are currently 44 empty tracks
There are now 0 empty tracks


It seems that some of the tracks didn't make it through the VAD because they were too quiet. I’ll drop these entries because if the VAD couldn’t pick up the audio, I doubt the model will.

# Save Files
Everything looks good so I'm going to save all the clean audio files.

In [7]:
filenames=pd.DataFrame()
count = 1
for index in data.index:
    entry = data.loc[index]
    # Save each file with the legend index as the new filename
    new_name = 'Clip_%04d.wav' % (index)
    path = 'Data/Clean Audio/'+new_name
    filenames = filenames.append([[entry['original'],new_name]])
    write_wave(path,entry['audio'],entry['sample rate'])
    # progress bar
    print('%4d of %d complete (%d%%)' % (count,len(data.index),(count/len(data.index))*100),end='\r')
    count+=1

Overall, I started with 8877 audio clips and, after the wrangling and filtering, only 8642 remain. Now to save the new filenames in the legend.

In [8]:
# organize filenames
filenames.columns = ['original','filename']
# and merge on the old names
new_legend = legend.merge(filenames,how='left',on='original')

new_legend.head()

,path,original,statment,emotion,sex,filename
0,Data/Raw Audio/CREMA_D/,1001_DFA_ANG_XX.wav,3,anger,Male,Clip_0000.wav
1,Data/Raw Audio/CREMA_D/,1001_DFA_DIS_XX.wav,3,disgust,Male,Clip_0001.wav
2,Data/Raw Audio/CREMA_D/,1001_DFA_FEA_XX.wav,3,fear,Male,Clip_0002.wav
3,Data/Raw Audio/CREMA_D/,1001_DFA_HAP_XX.wav,3,happy,Male,Clip_0003.wav
4,Data/Raw Audio/CREMA_D/,1001_DFA_NEU_XX.wav,3,neutral,Male,Clip_0004.wav


In [9]:
# perfect now save!
new_legend.to_csv('Data/CSVs/Audio Legend Clean.csv')

### Example Clip
Play a random audio clip from the dataset

In [10]:
# select a random clip
n = np.random.permutation(data.index)[0]
example = data.loc[n]

# print info about the clip
print(example.loc[['original','statment','emotion','sex']])

# play audio
example_audio = byte_to_float(example['audio'])
example_srate = example['sample rate']
sd.play(example_audio, example_srate)
status = sd.wait()

original    1026_IOM_NEU_XX.wav
statment                      5
emotion                 neutral
sex                        Male
Name: 2057, dtype: object
